# Infinite Craft with AI

My son was telling me about a game where he could put things together and that would produce a new thing and it was infinite. So I thought this would be a good experiment with LLM.

1. Start with a list of 4 things - fire, water, air and earth
2. When any 2 things are put together, create another thing by following these rules :
    - Do a lookup in a database. If the combination of those two things already exists, then create the third thing per what is in the db.
    - If that combo does not yet exist, use a LLM to create the new thing and store in the database. Then add that new thing to the list of things that the person can craft with.

## Create the database

In [1]:
import duckdb

def addItem(name):
    conn = duckdb.connect("things.db")
    conn.sql("INSERT INTO things VALUES (nextval('seq_thingid'), '" + name +"')")

# create a connection to a file called 'file.db'
con = duckdb.connect("things.db")

# create tables
con.sql("CREATE TABLE things (Thingid INTEGER PRIMARY KEY, Name VARCHAR)")
con.sql("CREATE TABLE combos (combo VARCHAR PRIMARY KEY, Name VARCHAR)")
con.sql("CREATE SEQUENCE seq_thingid START 1")
con.close()

addItem('fire')
addItem('water')
addItem('air')
addItem('earth')

In [22]:
con = duckdb.connect("things.db")
con.sql("SELECT * FROM things").show()
con.close()

┌─────────┬─────────┐
│ Thingid │  Name   │
│  int32  │ varchar │
├─────────┼─────────┤
│       1 │ fire    │
│       2 │ water   │
│       3 │ air     │
│       4 │ earth   │
└─────────┴─────────┘



## Initialize the LLM

In [3]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

In [4]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [7]:
template = ("Things 1: {word1}, Thing 2: {word2} "
            "Answer: word association. Given those two things, Thing 1 and Thing 2, " 
            "here is the most likely Thing that I can think of with less than 3 words. "
            "Here is just that thin with no other text, no matter how odd that thing may " 
            "be in three words or less and no ending punctuation: ")

prompt = PromptTemplate.from_template(template)

In [8]:
llm = OpenAI()
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [9]:
llm_chain.invoke({"word1":"beans", "word2":"salsa"})

{'word1': 'beans', 'word2': 'salsa', 'text': '\nBean dip.'}

In [62]:
def craft(thing1, thing2):
    thing1 = thing1.lower()
    thing2 = thing2.lower()
    thing3 = ""

    con = duckdb.connect("things.db")
    con.execute("SELECT * FROM things where Name = '" + thing1 + "'")
    result1 = con.fetchone()
    con.execute("SELECT * FROM things where Name = '" + thing2 + "'")
    result2 = con.fetchone()

    if(result1 == None or result2 == None):
        print("your things must exist")
    else:
        con.execute("SELECT Name from combos WHERE combo = '" + (thing1 + thing2) + "'")
        result = con.fetchone();
        if(result == None):
            resp = llm_chain.invoke({"word1":thing1, "word2":thing2})
            thing3 = resp["text"].lower().strip().replace('.', '').replace("\"", "").replace("'","")
            print("you are first to craft: " + thing3)
            print("adding to inventory")

            con.sql("INSERT INTO things VALUES (nextval('seq_thingid'), '" + thing3 +"')")
            con.sql("INSERT INTO combos VALUES ('" + (thing1 + thing2) + "', '" + thing3 + "')")
            print("ineserted")
        else:
            print(result[0])
    con.close()

In [65]:
craft("earths lifeline", "ocean currents")

you are first to craft: marine circulation
adding to inventory
ineserted


In [66]:
con = duckdb.connect("things.db")
con.sql("SELECT * FROM things").show()
con.sql("SELECT * FROM combos").show()
con.close()

┌─────────┬───────────────────────────┐
│ Thingid │           Name            │
│  int32  │          varchar          │
├─────────┼───────────────────────────┤
│       1 │ fire                      │
│       2 │ water                     │
│       3 │ air                       │
│       4 │ earth                     │
│       5 │ steam explosion.          │
│       6 │ elemental forces          │
│       7 │ "inferno fury"            │
│       8 │ hot wind gust             │
│       9 │ firestorm                 │
│      10 │ fire and earth            │
│      11 │ volcanic eruption         │
│      12 │ disaster duo              │
│      13 │ eruption aftermath chaos  │
│      14 │ tsunami catastrophe       │
│      15 │ ocean                     │
│      16 │ blue planet               │
│      17 │ beautiful, fragile orb    │
│      18 │ ocean, aquatic, or marine │
│      19 │ water extinguishes fire   │
│      20 │ water cycle               │
│      21 │ ocean currents            │
